In [70]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import branca
import folium
import json
import requests
from folium.plugins import MarkerCluster

import altair as alt
alt.renderers.enable('notebook')
from vega_datasets import data

from math import pi

pd.set_option('display.max_rows', 500)

url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
state_geo = f'{url}/us-states.json'
state_unemployment = f'{url}/US_Unemployment_Oct2012.csv'
state_data = pd.read_csv(state_unemployment)

# code reference at blog.prototypr.io/interactive-maps-with-python-part-1-aa1563dbe5a9
# transit districts source: www1.nyc.gov/assets/nypd/downloads/pdf/maps/transit-district-map.pdf

In [71]:
folium_map = folium.Map(location=[40.738, -73.98],
                        zoom_start=13,
                        tiles="openstreetmap")


districtGeoJson = open('districts.json')
#print(districtGeoJson)
districts = open('./nyc-maps-master/community_districts.geojson')
print(districts)
# add different map tiles
#folium.TileLayer('stamenterrain',attr='TEXT').add_to(folium_map)

# type of graph in 'json' format
json1_file = open('stackedBar.json')
json1_str = json1_file.read()
folium.CircleMarker(location=([40.738, -73.98]),
                            radius=10,
                            color="#0A8A9F",
                            popup=folium.Popup(max_width=550).add_child(
        folium.VegaLite(json1_str, width=550, height=250)),
                            fill=True).add_to(folium_map)

folium.Choropleth(
    geo_data=state_geo,
    name='choropleth',
    data=state_data,
    columns=['State', 'Unemployment'],
    key_on='feature.id',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Unemployment Rate (%)'
).add_to(folium_map)

folium.LayerControl().add_to(folium_map)
folium_map

<_io.TextIOWrapper name='./nyc-maps-master/community_districts.geojson' mode='r' encoding='UTF-8'>


In [72]:
# load csv of all station information
subway_data = pd.read_csv("Stations.csv")

# need to get lat/long of all stations to draw circle
# need to figure out how to make interactable (default pics)
subway_data

,Station ID,Complex ID,GTFS Stop ID,Division,Line,Stop Name,Borough,Daytime Routes,Structure,GTFS Latitude,GTFS Longitude,North Direction Label,South Direction Label
0,1,1,R01,BMT,Astoria,Astoria - Ditmars Blvd,Q,N W,Elevated,40.775036,-73.912034,NaN,Manhattan
1,2,2,R03,BMT,Astoria,Astoria Blvd,Q,N W,Elevated,40.770258,-73.917843,Ditmars Blvd,Manhattan
2,3,3,R04,BMT,Astoria,30 Av,Q,N W,Elevated,40.766779,-73.921479,Astoria - Ditmars Blvd,Manhattan
3,4,4,R05,BMT,Astoria,Broadway,Q,N W,Elevated,40.761820,-73.925508,Astoria - Ditmars Blvd,Manhattan
4,5,5,R06,BMT,Astoria,36 Av,Q,N W,Elevated,40.756804,-73.929575,Astoria - Ditmars Blvd,Manhattan
5,6,6,R08,BMT,Astoria,39 Av,Q,N W,Elevated,40.752882,-73.932755,Astoria - Ditmars Blvd,Manhattan
6,7,613,R11,BMT,Astoria,Lexington Av/59 St,M,N W R,Subway,40.762660,-73.967258,Queens,Downtown & Brooklyn
7,8,8,R13,BMT,Astoria,5 Av/59 St,M,N W R,Subway,40.764811,-73.973347,Queens,Downtown & Brooklyn
8,9,9,R14,BMT,Broadway - Brighton,57 St - 7 Av,M,N Q R W,Subway,40.764664,-73.980658,Uptown & Queens,Downtown & Brooklyn
9,10,10,R15,BMT,Broadway - Brighton,49 St,M,N R W,Subway,40.759901,-73.984139,Uptown & Queens,Downtown & Brooklyn


In [73]:
locations = subway_data.groupby("GTFS Latitude").first()
locations = locations.loc[:, ["GTFS Longitude",
                              "Stop Name",
                              "Borough"]]
locations.groupby('Stop Name')
locations.head()
locations.drop_duplicates(subset ='Stop Name',keep='first',inplace=True)
locations.sort_values(by='Stop Name')

,GTFS Longitude,Stop Name,Borough
GTFS Latitude,,,
40.730953,-73.981628,1 Av,M
40.790600,-73.947478,103 St,M
40.749865,-73.862700,103 St - Corona Plaza,Q
40.681711,-73.837683,104 St,Q
40.795020,-73.944250,110 St,M
40.684331,-73.832163,111 St,Q
40.798629,-73.941617,116 St,M
40.807722,-73.964110,116 St - Columbia University,M
40.700492,-73.828294,121 St,Q


In [74]:
highCountStations = ['59 St - Columbus Circle','14 St','125 St','86 St','167 St','Jackson Av','Court Sq','Rockaway Blvd','Franklin Av','Atlantic Av','Utica Av','Newkirk Av']
highCountStations.sort()
lowCountStations = ['72 St','Franklin St','157 St','51 St','Mt Eden Av','Freeman St','Astoria Blvd','Beach 98 St','Carroll St','Winthrop St','Norwood Av','Ditmas Av']


In [83]:
highLocations = locations[locations['Stop Name'].isin(highCountStations)]
highSpots = ['High','High','High','High','High','High','High','High','High','High','High','High']
highLocations.sort_values(by='Stop Name')
highLocations['Cop Sightings'] = highSpots
highLocations

/Users/dylanfosgett/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,GTFS Longitude,Stop Name,Borough,Cop Sightings
GTFS Latitude,,,,
40.592721,-73.978230,86 St,Bk,High
40.639967,-73.948411,Newkirk Av,Bk,High
40.670682,-73.958131,Franklin Av,Bk,High
40.675345,-73.903097,Atlantic Av,Bk,High
40.679364,-73.930729,Utica Av,Bk,High
40.680429,-73.843853,Rockaway Blvd,Q,High
40.737826,-74.000201,14 St,M,High
40.746554,-73.943832,Court Sq,Q,High
40.768247,-73.981929,59 St - Columbus Circle,M,High


In [84]:
lowCountStations.sort()
lowCountStations

['157 St',
 '51 St',
 '72 St',
 'Astoria Blvd',
 'Beach 98 St',
 'Carroll St',
 'Ditmas Av',
 'Franklin St',
 'Freeman St',
 'Mt Eden Av',
 'Norwood Av',
 'Winthrop St']

In [85]:
lowLocations = locations[locations['Stop Name'].isin(lowCountStations)]
lowLocations.sort_values(by='Stop Name')
lowSpots = ['Low','Low','Low','Low','Low','Low','Low','Low','Low','Low','Low','Low']
lowLocations['Cop Sightings'] = lowSpots
lowLocations

/Users/dylanfosgett/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,GTFS Longitude,Stop Name,Borough,Cop Sightings
GTFS Latitude,,,,
40.585307,-73.820558,Beach 98 St,Q,Low
40.636119,-73.978172,Ditmas Av,Bk,Low
40.656652,-73.950200,Winthrop St,Bk,Low
40.680303,-73.995048,Carroll St,Bk,Low
40.681410,-73.880039,Norwood Av,Bk,Low
40.719318,-74.006886,Franklin St,M,Low
40.757107,-73.971920,51 St,M,Low
40.768799,-73.958424,72 St,M,Low
40.770258,-73.917843,Astoria Blvd,Q,Low


In [86]:
transitDistricts = [3,2,3,11,4,1,1,1,20,32,23,30,30,34,32,2,12,12,11,32,33,23,33,32]

In [87]:
lowHighLocations = [lowLocations,highLocations]
lowHigh = pd.concat(lowHighLocations)
lowHigh = lowHigh.sort_values(by='Stop Name',ascending=True)
lowHigh['Transit District'] = transitDistricts
lowHigh.head()

,GTFS Longitude,Stop Name,Borough,Cop Sightings,Transit District
GTFS Latitude,,,,,
40.804138,-73.937594,125 St,M,High,3
40.737826,-74.000201,14 St,M,High,2
40.834041,-73.944890,157 St,M,Low,3
40.833771,-73.918440,167 St,Bx,High,11
40.757107,-73.971920,51 St,M,Low,4


In [88]:
def pie_chart_maker(randInt):
    # Pie chart, where the slices will be ordered and plotted counter-clockwise:
    labels = 'one','two'
    #colors = 'orange','gray'
    sizes = [randInt,(1-randInt)]
    explode = (0,0)

    fig1, ax1 = plt.subplots()
    ax1.pie(sizes, explode=explode,
            shadow=True, startangle=90)
    ax1.legend(labels, loc='best',
               fontsize=10)

    #ax1.rcParams["figure.figsize"] = [4,3]

    #ax1.pie(sizes, labels,autopct='%1.1f%%')
    ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
    
    return ax1

    #plt.show()

In [89]:
def make_pie():    
    output_file("pie.html")

    x = {
        'United States': 157,
        'United Kingdom': 93,
        'Japan': 89,
        'China': 63,
        'Germany': 44,
        'India': 42,
        'Italy': 40,
        'Australia': 35,
        'Brazil': 32,
        'France': 31,
        'Taiwan': 31,
        'Spain': 29
    }

    data = pd.Series(x).reset_index(name='value').rename(columns={'index':'country'})
    data['angle'] = data['value']/data['value'].sum() * 2*pi
    data['color'] = Category20c[len(x)]

    p = figure(plot_height=350, title="Pie Chart", toolbar_location=None,
               tools="hover", tooltips="@country: @value", x_range=(-0.5, 1.0))

    p.wedge(x=0, y=1, radius=0.4,
            start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
            line_color="white", fill_color='color', legend_field='country', source=data)

    p.axis.axis_label=None
    p.axis.visible=False
    p.grid.grid_line_color = None
    
    return show(p)
    #show(p)

In [90]:
chart = alt.Chart(data.cars.url).mark_point().encode(
    x='Horsepower:Q',
    y='Miles_per_Gallon:Q',
    color='Origin:N'
)

chart.save('chart.json')

In [91]:
# plot circles at each station
def plot_stations(station):
    folium_map = folium.Map(location=[40.738, -73.98],
                            zoom_start=10,
                            tiles="openstreetmap",
                            width='100%')
    
    # for each row in the data, add a circle marker
    for index, row in station.iterrows():
        #index = latitude, 
        #row["GTFS Longitude"] = longitude
        
        #print(row.isin(lowLocations['Stop Name']))
        # number of spotted officers plotted
        
        # generate popup message shown on click
        popup_text = "stop name: {}"
        popup_text = popup_text.format(row["Stop Name"])
        #popup_text = json1_str
        #popup_text = folium.Popup(max_width=550).add_child(
                                  #folium.VegaLite(json1_str, width=550, height=250))
        
        # radius of circles
        radius = 10
        # color of circles (can change conditionally, by borough)
        if(row["Borough"] == 'Bx'):
            #Bronx = red
            color='#FF0000'
        elif(row["Borough"] == 'Bk'):
            #Brooklyn=yellow
            color='#ffff00'
        elif(row["Borough"] == 'M'):
            #Manhattan=green
            color='#00ff00'
        elif(row["Borough"] == 'Q'):
            #Queens=orange
            color='#ffa500'
        elif(row["Borough"] == 'SI'):
            #Staten Island=Purple
            color="#0A8A9F"
        else:
            #Missing Borough colored blue
            color='#800080'
        # color for low Locations is: purple
        for i in (lowLocations['Stop Name']):
            if(row['Stop Name'] == i):
                color = '#0A8A9F'
        for i in (highLocations['Stop Name']):
            if(row['Stop Name'] == i):
                color = '#8b0000'
        #print(color)
        #color='#00ff00'
        folium.CircleMarker(location=(index,
                                      row["GTFS Longitude"]),
                            radius=radius,
                            color=color,
                            popup=popup_text,
                            fill=False,
                            fill_color=color,
                            tooltip=popup_text,
                            clustered_marker=True).add_to(folium_map)

    return folium_map

In [92]:
# plot circles at each station
# colored by specific stations
def plot_stations_colored_by_name(station):
    folium_map = folium.Map(location=[40.738, -73.98],
                            zoom_start=10,
                            tiles="openstreetmap",
                            width='100%')
    folium.TileLayer('stamentoner').add_to(folium_map)
    folium.TileLayer('stamenterrain').add_to(folium_map)
    folium.LayerControl().add_to(folium_map)
    
    # for each row in the data, add a circle marker
    for index, row in station.iterrows():
        #index = latitude, 
        #row["GTFS Longitude"] = longitude
        
        #print(row.isin(lowLocations['Stop Name']))
        # number of spotted officers plotted
        
        # generate popup message shown on click
        #popup_text = "stop name: {}"
        #popup_text = popup_text.format(row["Stop Name"])
        popup_text = json1_str
        popup_text = folium.Popup(max_width=550).add_child(
                                  folium.VegaLite(json1_str, width=550, height=250))
        
        # tooltip is text displayed when only hovering over popup
        tooltip_text = "{}"
        tooltip_text = tooltip_text.format(row["Stop Name"])
        # radius of circles
        radius = 10
        # color of circles (can change conditionally, by borough)
        #if(row["Borough"] == 'Bx'):
            #Bronx = red
            #color='#FF0000'
        #elif(row["Borough"] == 'Bk'):
            #Brooklyn=yellow
            #color='#ffff00'
        #elif(row["Borough"] == 'M'):
            #Manhattan=green
            #color='#00ff00'
        #elif(row["Borough"] == 'Q'):
            #Queens=orange
            #color='#ffa500'
        #elif(row["Borough"] == 'SI'):
            #Staten Island=Purple
            #color="#0A8A9F"
        #else:
            #Missing Borough colored blue
            #color='#800080'
        # color for low Locations is: purple
        for i in (lowLocations['Stop Name']):
            if(row['Stop Name'] == i):
                color = '#008b46'
        for i in (highLocations['Stop Name']):
            if(row['Stop Name'] == i):
                color = '#8b0000'
        #print(color)
        #color='#00ff00'
        folium.CircleMarker(location=(index,
                                      row["GTFS Longitude"]),
                            radius=radius,
                            color=color,
                            popup=popup_text,
                            fill=False,
                            fill_color=color,
                            tooltip=tooltip_text,
                            clustered_marker=True).add_to(folium_map)

    return folium_map

In [93]:
# plot circles at each station
# colored by specific stations
def plot_stations_colored_by_transit_district(station):
    folium_map = folium.Map(location=[40.738, -73.98],
                            zoom_start=10,
                            tiles="stamentoner",
                            width='100%')
    folium.TileLayer('openStreetMap').add_to(folium_map)
    folium.TileLayer('stamenterrain').add_to(folium_map)
    folium.LayerControl().add_to(folium_map)
    
    # for each row in the data, add a circle marker
    for index, row in station.iterrows():
        #index = latitude, 
        #row["GTFS Longitude"] = longitude
        
        #print(row.isin(lowLocations['Stop Name']))
        # number of spotted officers plotted
        
        # generate popup message shown on click
        #popup_text = "stop name: {}"
        #popup_text = popup_text.format(row["Stop Name"])
        popup_text = json1_str
        popup_text = folium.Popup(max_width=550).add_child(
                                  folium.VegaLite(json1_str, width=550, height=250))
        
        # tooltip is text displayed when only hovering over popup
        tooltip_text = "Stop Name: {}<br> Transit District: {}<br> Police Sightings: {}<br>"
        tooltip_text = tooltip_text.format(row["Stop Name"], row["Transit District"],row["Cop Sightings"])
        # radius of circles
        radius = 10
        # color of circles (can change conditionally)
        
        # color for low Locations is: purple
        if(row['Transit District']==3):
            #brown
            color = '#964b00'
        if(row['Transit District']==2):
            #green
            color = '#00ff00'
        if(row['Transit District']==11):
            #pink
            color = '#ff69b4'
        if(row['Transit District']==4):
            #black
            color = '#000000'
        if(row['Transit District']==1):
            #red
            color = '#ff0000'
        if(row['Transit District']==20):
            #blue
            color = '#0000ff'
        if(row['Transit District']==32):
            #violet
            color = '#8a2be2'
        if(row['Transit District']==23):
            #orange
            color = '#ffa500'
        if(row['Transit District']==30):
            #teal
            color = '#008080'
        if(row['Transit District']==34):
            #brown
            color = '#b5651d'
        if(row['Transit District']==12):
            #yellow
            color = '#ffff00'
        if(row['Transit District']==33):
            #purple
            color = '#800080'
        
        #print(color)
        #color='#00ff00'
        folium.CircleMarker(location=(index,
                                      row["GTFS Longitude"]),
                            radius=radius,
                            color=color,
                            popup=popup_text,
                            fill=True,
                            fill_color=color,
                            fill_opacity=1,
                            tooltip=tooltip_text,
                            clustered_marker=True).add_to(folium_map)

    return folium_map

In [94]:
# call function plot_stations of each subway stop
#plot_stations(locations)

In [95]:
# CREATES MAIN MAP
plot_stations_colored_by_transit_district(lowHigh)

In [97]:
folium_map = plot_stations_colored_by_transit_district(lowHigh)

In [98]:
#folium_map.save("ClickableSubwayMapStackedBarColoredTransitDistrict.html")

In [17]:
#folium_map.save("testGeoJson.html")